# Extract only good items and good users

In [1]:
import os
import sys
import json
import datetime
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

from tqdm import tqdm
from collections import defaultdict, Counter
import os
import sys
import json
import datetime
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

from tqdm import tqdm
from collections import defaultdict, Counter

%matplotlib inline

In [2]:
relative_path_to_data = ".\\data\\"

path_to_df = relative_path_to_data + r'ratings_Beauty.csv'
# df = pd.read_csv(path_to_df)
df = pd.read_csv(path_to_df, names=['user_id', 'item_id', 'rating', 'timestamp'])
print(df.shape)
print(relative_path_to_data)

(2023070, 4)
.\data\


In [3]:
df.head()

,user_id,item_id,rating,timestamp
0,A39HTATAQ9V7YF,0205616461,5.0,1369699200
1,A3JM6GV9MNOF9X,0558925278,3.0,1355443200
2,A1Z513UWSAAO0F,0558925278,5.0,1404691200
3,A1WMRR494NWEWV,0733001998,4.0,1382572800
4,A3IAAVS479H7M7,0737104473,1.0,1274227200


In [4]:
df.isnull().sum()

user_id      0
item_id      0
rating       0
timestamp    0
dtype: int64

In [5]:
df.user_id.max(), df.user_id.unique().shape

('AZZZU2TD7Q3ET', (1210271,))

In [6]:
df.user_id = pd.factorize(df.user_id)[0] + 1
df.user_id.min(), df.user_id.max(), df.user_id.unique().shape

(np.int64(1), np.int64(1210271), (1210271,))

In [7]:
df.item_id = pd.factorize(df.item_id)[0] + 1
df.item_id.min(), df.item_id.max(), df.item_id.unique().shape

(np.int64(1), np.int64(249274), (249274,))

In [8]:
df.head()

,user_id,item_id,rating,timestamp
0,1,1,5.0,1369699200
1,2,2,3.0,1355443200
2,3,2,5.0,1404691200
3,4,3,4.0,1382572800
4,5,4,1.0,1274227200


In [9]:
data = []

for _, row in tqdm(df.iterrows()):
    data.append({
        'user_id': int(row.user_id),
        'item_id': int(row.item_id),
        'timestamp': int(row.timestamp)
    })

print(len(data))

2023070it [00:35, 57381.44it/s]

2023070


In [10]:
user_history = defaultdict(list)
item_history = defaultdict(list)

for row in tqdm(data):
    user_raw_id = row['user_id']
    item_raw_id = row['item_id']
    interaction_timestamp = row['timestamp']
    
    user_history[user_raw_id].append({'item_id': item_raw_id, 'timestamp': interaction_timestamp})
    item_history[item_raw_id].append({'user_id': user_raw_id, 'timestamp': interaction_timestamp})

is_changed = True
threshold = 5
good_users = set()
good_items = set()


while is_changed:
    old_state = (len(good_users), len(good_items))
    
    good_users = set()
    good_items = set()

    for user_id, history in user_history.items():
        if len(history) >= threshold:
            good_users.add(user_id)

    for item_id, history in item_history.items():
        if len(history) >= threshold:
            good_items.add(item_id)
    
    user_history = {
        user_id: list(filter(lambda x: x['item_id'] in good_items, history))
        for user_id, history in user_history.items()
    }
    
    item_history = {
        item_id: list(filter(lambda x: x['user_id'] in good_users, history))
        for item_id, history in item_history.items()
    }
    
    new_state = (len(good_users), len(good_items))
    is_changed = (old_state != new_state)
    print(old_state, new_state)

100%|████████████████████████████████████████████████████████████████████| 2023070/2023070 [00:02<00:00, 765977.82it/s]


(0, 0) (52374, 67345)
(52374, 67345) (40226, 19369)
(40226, 19369) (27501, 17041)
(27501, 17041) (26116, 13727)
(26116, 13727) (23746, 13318)
(23746, 13318) (23436, 12562)
(23436, 12562) (22787, 12458)
(22787, 12458) (22705, 12247)
(22705, 12247) (22505, 12224)
(22505, 12224) (22480, 12153)
(22480, 12153) (22408, 12140)
(22408, 12140) (22401, 12116)
(22401, 12116) (22374, 12114)
(22374, 12114) (22372, 12103)
(22372, 12103) (22364, 12103)
(22364, 12103) (22364, 12101)
(22364, 12101) (22363, 12101)
(22363, 12101) (22363, 12101)


In [11]:
user_mapping = {}
item_mapping = {}
tmp_user_history = defaultdict(list)
tmp_item_history = defaultdict(list)

for user_id, history in tqdm(user_history.items()):
    processed_history = []

    for filtered_item in history:
        item_id = filtered_item['item_id']
        item_timestamp = filtered_item['timestamp']

        processed_item_id = item_mapping.get(item_id, len(item_mapping) + 1)
        item_mapping[item_id] = processed_item_id

        processed_history.append({'item_id': processed_item_id, 'timestamp': item_timestamp})
        
    if len(processed_history) >= threshold:
        processed_user_id = user_mapping.get(user_id, len(user_mapping) + 1)
        user_mapping[user_id] = processed_user_id

        tmp_user_history[processed_user_id] = sorted(processed_history, key=lambda x: x['timestamp'])

    
for item_id, history in tqdm(item_history.items()):
    processed_history = []

    for filtered_user in history:
        user_id = filtered_user['user_id']
        user_timestamp = filtered_user['timestamp']

        processed_user_id = user_mapping.get(user_id, len(user_mapping) + 1)
        user_mapping[user_id] = processed_user_id

        processed_history.append({'user_id': processed_user_id, 'timestamp': user_timestamp})

    if len(processed_history) >= threshold:
        processed_item_id = item_mapping.get(item_id, len(item_mapping) + 1)
        item_mapping[item_id] = processed_item_id

        tmp_item_history[processed_item_id] = sorted(processed_history, key=lambda x: x['timestamp'])

user_history = tmp_user_history
item_history = tmp_item_history

100%|██████████████████████████████████████████████████████████████████████| 249274/249274 [00:00<00:00, 958099.56it/s]


In [12]:
print('Users count:', len(user_mapping))
print('Items count:', len(item_mapping))
print('Actions count:', sum(list(map(lambda x: len(x), user_history.values()))))
print('Avg user history len:', np.mean(list(map(lambda x: len(x), user_history.values()))))
print('Avg item history len:', np.mean(list(map(lambda x: len(x), item_history.values()))))

Users count: 22363
Items count: 12101
Actions count: 198502
Avg user history len: 8.876358270357287
Avg item history len: 16.403768283612923


# Getting test and valifation timelines by ration

In [13]:
# List to collect all items
all_items = []
# Iterate over all users and their respective item lists
for user_id, items in user_history.items():
    # Iterate over each item in the user's history
    for item in items:
        # Append each item to the all_items list
        all_items.append(item)

# Sort all_items by the 'timestamp' field in ascending order
all_items.sort(key=lambda x: x['timestamp'])

print(len(all_items))

# Print the sorted list of items
print(all_items[:5])

198502
[{'item_id': 3133, 'timestamp': 1023840000}, {'item_id': 3438, 'timestamp': 1024185600}, {'item_id': 3439, 'timestamp': 1024185600}, {'item_id': 4263, 'timestamp': 1036627200}, {'item_id': 3436, 'timestamp': 1052611200}]


In [14]:
ratio_validation=0.1
ratio_test = 0.1


test_item_idx_border = int(len(all_items) - len(all_items) * ratio_test)
validation_item_idx_border = int(test_item_idx_border - len(all_items) * ratio_validation)

print(validation_item_idx_border)
print(test_item_idx_border)

timeline_validation = all_items[validation_item_idx_border]['timestamp']
timeline_test = all_items[test_item_idx_border]['timestamp']


# timilime_validation <= timelime_test
print(timeline_validation)
print(timeline_test)

158800
178651
1394150400
1399939200


In [15]:
# check
count_lesser_than_timeline_validation = 0
count_lesser_than_timeline_test = 0

for item in all_items:
    if item['timestamp'] < timeline_validation:
        count_lesser_than_timeline_validation+=1
    if item['timestamp'] < timeline_test:
        count_lesser_than_timeline_test+=1

# border checking
print(count_lesser_than_timeline_validation)
print(count_lesser_than_timeline_test)
print()
print(all_items[count_lesser_than_timeline_validation-1])
print(all_items[count_lesser_than_timeline_validation])
print(all_items[validation_item_idx_border])
print()
print(all_items[count_lesser_than_timeline_test-1])
print(all_items[count_lesser_than_timeline_test])
print(all_items[test_item_idx_border])

158689
178355

{'item_id': 12100, 'timestamp': 1394064000}
{'item_id': 373, 'timestamp': 1394150400}
{'item_id': 7548, 'timestamp': 1394150400}

{'item_id': 2378, 'timestamp': 1399852800}
{'item_id': 3235, 'timestamp': 1399939200}
{'item_id': 1950, 'timestamp': 1399939200}


# Prepare all_data.txt

In [16]:
import queue

In [17]:
min_window_size = 5
window_size = 256

bert4rec_data_train = [] #TODO: shuffle data after collection
bert4rec_data_validation = []
bert4rec_data_test = []

q = queue.Queue()

for user_id, items in user_history.items():
    items = sorted(items, key=lambda x: x['timestamp'])
                   
    q.queue.clear()
    if len(items) < min_window_size:
       continue
    
    for idx, item in enumerate(items):
        # queue.push(item)
        q.put(item)
        if len(q.queue) > window_size:
            q.get()
 
        if idx >= min_window_size:

            last_element = q.queue[-1]
            window_elemnts = list(q.queue)

            if last_element['timestamp'] < timeline_validation:
                bert4rec_data_train.append({user_id: window_elemnts})
            elif last_element['timestamp'] < timeline_test:
                bert4rec_data_validation.append({user_id: window_elemnts})
            else:
                bert4rec_data_test.append({user_id: window_elemnts})

In [18]:
for sample in bert4rec_data_train[:5]:
    print(sample)

print(len(bert4rec_data_train))
print(len(bert4rec_data_validation))    
print(len(bert4rec_data_test))

{1: [{'item_id': 9, 'timestamp': 1214956800}, {'item_id': 10, 'timestamp': 1216684800}, {'item_id': 11, 'timestamp': 1218067200}, {'item_id': 12, 'timestamp': 1218067200}, {'item_id': 13, 'timestamp': 1219881600}, {'item_id': 16, 'timestamp': 1222387200}]}
{1: [{'item_id': 9, 'timestamp': 1214956800}, {'item_id': 10, 'timestamp': 1216684800}, {'item_id': 11, 'timestamp': 1218067200}, {'item_id': 12, 'timestamp': 1218067200}, {'item_id': 13, 'timestamp': 1219881600}, {'item_id': 16, 'timestamp': 1222387200}, {'item_id': 14, 'timestamp': 1224806400}]}
{1: [{'item_id': 9, 'timestamp': 1214956800}, {'item_id': 10, 'timestamp': 1216684800}, {'item_id': 11, 'timestamp': 1218067200}, {'item_id': 12, 'timestamp': 1218067200}, {'item_id': 13, 'timestamp': 1219881600}, {'item_id': 16, 'timestamp': 1222387200}, {'item_id': 14, 'timestamp': 1224806400}, {'item_id': 15, 'timestamp': 1227830400}]}
{1: [{'item_id': 9, 'timestamp': 1214956800}, {'item_id': 10, 'timestamp': 1216684800}, {'item_id': 11,

In [19]:
with open(relative_path_to_data + r'bert4rec_data_train.txt', 'w') as f:
    for sample in bert4rec_data_train:
        for user_id, item_history in sample.items():
            f.write(' '.join([str(user_id)] + [
                str(item_event['item_id']) for item_event in sorted(item_history, key=lambda x: x['timestamp'])
            ]))
            f.write('\n')

with open(relative_path_to_data + r'bert4rec_data_validation.txt', 'w') as f:
    for sample in bert4rec_data_validation:
        for user_id, item_history in sample.items():
            f.write(' '.join([str(user_id)] + [
                str(item_event['item_id']) for item_event in sorted(item_history, key=lambda x: x['timestamp'])
            ]))
            f.write('\n')

with open(relative_path_to_data + r'bert4rec_data_test.txt', 'w') as f:
    for sample in bert4rec_data_test:
        for user_id, item_history in sample.items():
            f.write(' '.join([str(user_id)] + [
                str(item_event['item_id']) for item_event in sorted(item_history, key=lambda x: x['timestamp'])
            ]))
            f.write('\n')

In [20]:
sasrec_data_train = [] #TODO: shuffle data after collection
sasrec_data_validation = bert4rec_data_validation
sasrec_data_test = bert4rec_data_test

for user_id, items in user_history.items():
    items = sorted(items, key=lambda x: x['timestamp'])
                   
    if len(items) < min_window_size:
       continue

    window_elements = []
    
    for idx, item in enumerate(items):
        if item['timestamp'] >= timeline_validation:
            break
        window_elements.append(item)
        
        if len(window_elements) >= window_size:
            sasrec_data_train.append({user_id: window_elements})
            window_elements = []

    if len(window_elements) >= min_window_size:
        sasrec_data_train.append({user_id: window_elements})

In [21]:
for sample in sasrec_data_train[:5]:
    print(sample)
    print()

print(len(sasrec_data_train))
print(len(sasrec_data_validation))    
print(len(sasrec_data_test))

{1: [{'item_id': 9, 'timestamp': 1214956800}, {'item_id': 10, 'timestamp': 1216684800}, {'item_id': 11, 'timestamp': 1218067200}, {'item_id': 12, 'timestamp': 1218067200}, {'item_id': 13, 'timestamp': 1219881600}, {'item_id': 16, 'timestamp': 1222387200}, {'item_id': 14, 'timestamp': 1224806400}, {'item_id': 15, 'timestamp': 1227830400}, {'item_id': 4, 'timestamp': 1232323200}, {'item_id': 19, 'timestamp': 1232841600}, {'item_id': 20, 'timestamp': 1236643200}, {'item_id': 17, 'timestamp': 1237593600}, {'item_id': 18, 'timestamp': 1241136000}, {'item_id': 26, 'timestamp': 1275177600}, {'item_id': 24, 'timestamp': 1276128000}, {'item_id': 27, 'timestamp': 1276128000}, {'item_id': 25, 'timestamp': 1288828800}, {'item_id': 30, 'timestamp': 1297123200}, {'item_id': 33, 'timestamp': 1301097600}, {'item_id': 7, 'timestamp': 1320278400}, {'item_id': 29, 'timestamp': 1322438400}, {'item_id': 35, 'timestamp': 1327536000}, {'item_id': 36, 'timestamp': 1327536000}, {'item_id': 21, 'timestamp': 134

In [22]:
with open(relative_path_to_data + r'sasrec_data_train.txt', 'w') as f:
    for sample in sasrec_data_train:
        for user_id, item_history in sample.items():
            f.write(' '.join([str(user_id)] + [
                str(item_event['item_id']) for item_event in sorted(item_history, key=lambda x: x['timestamp'])
            ]))
            f.write('\n')

with open(relative_path_to_data + r'sasrec_data_validation.txt', 'w') as f:
    for sample in sasrec_data_validation:
        for user_id, item_history in sample.items():
            f.write(' '.join([str(user_id)] + [
                str(item_event['item_id']) for item_event in sorted(item_history, key=lambda x: x['timestamp'])
            ]))
            f.write('\n')

with open(relative_path_to_data + r'sasrec_data_test.txt', 'w') as f:
    for sample in sasrec_data_test:
        for user_id, item_history in sample.items():
            f.write(' '.join([str(user_id)] + [
                str(item_event['item_id']) for item_event in sorted(item_history, key=lambda x: x['timestamp'])
            ]))
            f.write('\n')